In [1]:
import pandas as pd

# Load the Parquet file
parquet_file_path = 'snp_rsi_macd.parquet'
data = pd.read_parquet(parquet_file_path)

# Save to a CSV file
csv_file_path = 'snp_rsi_macd.csv'
data.to_csv(csv_file_path, index=False)

In [2]:
import zipfile

def compress_csv_zip(input_file, output_zip):
    with zipfile.ZipFile(output_zip, 'w', zipfile.ZIP_DEFLATED) as zf:
        zf.write(input_file, arcname=input_file)

# Example usage
compress_csv_zip('snp_rsi_macd.csv', 'snp_rsi_macd.zip')

In [3]:
data = pd.read_csv('snp_rsi_macd.csv')

In [4]:
import pandas as pd
import numpy as np

# Helper functions
def EMA(prices, span=12, adjust=False):
    return prices.ewm(span=span, adjust=adjust).mean()

def MACD(prices, span1=12, span2=26, signal_span=9):
    fast_ema = EMA(prices, span=span1)
    slow_ema = EMA(prices, span=span2)
    macd = fast_ema - slow_ema
    signal = EMA(macd, span=signal_span)
    return macd.iloc[-1], signal.iloc[-1]  # Return the last value for simplicity

def RSI(prices, periods=14):
    delta = prices.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=periods).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=periods).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi.iloc[-1]  # Return the last value for simplicity

# Main function to process data and calculate indicators
def calculate_indicators(data):
    results = pd.DataFrame()
    # Ensure data has only the price columns by filtering non-numeric columns
    numeric_data = data.select_dtypes(include=[np.number])
    # Calculate MACD and RSI
    macd, signal = MACD(numeric_data)
    rsi = RSI(numeric_data)
    # Store results
    results = pd.DataFrame({
        'MACD': macd,
        'Signal': signal,
        'RSI': rsi
    })
    return results


# Ensure 'Stock' column is the index if it exists in the columns
if 'Stock' in data.columns:
    data = data.set_index('Stock')
elif 'Stock' not in data.index:
    print("No 'Stock' column found, please check your DataFrame structure.")

# Calculate MACD and RSI
indicator_results = calculate_indicators(data)

# Display or inspect the results
print(indicator_results.head())


                             MACD        Signal  RSI
2014-06-16 00:00:00 -5.616880e-62 -5.616880e-62  NaN
2014-06-17 00:00:00  3.367288e-02  3.367288e-02  NaN
2014-06-18 00:00:00  3.735695e-02  3.735695e-02  NaN
2014-06-19 00:00:00  3.039524e-02  3.039524e-02  NaN
2014-06-20 00:00:00  2.327547e-02  2.327547e-02  NaN


In [5]:
import pandas as pd

# Load your data
data = pd.read_csv('snp_rsi_macd.csv')

# Assuming the stock price columns are named as dates and the rest are metadata
price_columns = data.filter(regex=r'\d{4}-\d{2}-\d{2}').columns

# Functions to compute MACD and RSI
def compute_macd(data, fast_period=12, slow_period=26, signal_period=9):
    fast_ema = data.ewm(span=fast_period, adjust=False).mean()
    slow_ema = data.ewm(span=slow_period, adjust=False).mean()
    macd_line = fast_ema - slow_ema
    signal_line = macd_line.ewm(span=signal_period, adjust=False).mean()
    return macd_line.iloc[-1], signal_line.iloc[-1]

def compute_rsi(data, periods=14):
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).ewm(span=periods, adjust=False).mean()
    loss = (-delta.where(delta < 0, 0)).ewm(span=periods, adjust=False).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs)).iloc[-1]

# Calculate MACD and RSI for each stock
macd = []
signal = []
rsi = []
for index, row in data[price_columns].iterrows():
    macd_value, signal_value = compute_macd(row)
    rsi_value = compute_rsi(row)
    macd.append(macd_value)
    signal.append(signal_value)
    rsi.append(rsi_value)

# Add these values to the original dataframe
data['MACD'] = macd
data['Signal'] = signal
data['RSI'] = rsi

# Save the updated DataFrame to a new CSV file
data.to_csv('snp_beta_macd_signal_rsi.csv', index=False)

# Check the head of the updated DataFrame
print(data[['Stock', 'Industry', 'Beta', 'MACD', 'Signal', 'RSI']].head())


  Stock                      Industry   Beta      MACD    Signal        RSI
0     A        Diagnostics & Research  1.068 -3.767062 -2.842506  25.132294
1   AAL                      Airlines  1.498 -0.724258 -0.682215  25.170934
2  AAPL          Consumer Electronics  1.250  7.345075  5.599729  78.073405
3  ABBV  Drug Manufacturers - General  0.582  1.336143  0.417622  63.985570
4  ABNB               Travel Services  1.188 -1.365007 -1.966829  44.163364


In [ ]:
import zipfile

def compress_csv_zip(input_file, output_zip):
    with zipfile.ZipFile(output_zip, 'w', zipfile.ZIP_DEFLATED) as zf:
        zf.write(input_file, arcname=input_file)

# Example usage
compress_csv_zip('snp_beta_macd_signal_rsi.csv', 'snp_beta_macd_signal_rsi.zip')
